### Paso 0 : importe de librerias y configuraciond el entrono

In [2]:
import gdown
import glob
import utils
import random
import numpy as np
import torch
import os
import subprocess
import rastervision
import rasterio
from subprocess import check_output
from aux_functions import download_blob_from_azure

from rastervision.core.data import RasterioSource, MinMaxTransformer

from rastervision.core.data import (
    ClassConfig, GeoJSONVectorSource, RasterioCRSTransformer,
    RasterizedSource, ClassInferenceTransformer)

from rastervision.core.data import SemanticSegmentationLabelSource

from rastervision.core.data.utils.geojson import get_polygons_from_uris
from shapely.geometry import Polygon

from rastervision.pytorch_learner import (
    SemanticSegmentationRandomWindowGeoDataset, SemanticSegmentationSlidingWindowGeoDataset, SemanticSegmentationVisualizer)

import albumentations as A

import torch
from torch.utils.data import ConcatDataset

from rastervision.pytorch_learner import SemanticSegmentationGeoDataConfig
from rastervision.pytorch_learner import SolverConfig
from rastervision.pytorch_learner import SemanticSegmentationLearnerConfig
from rastervision.pytorch_learner import SemanticSegmentationLearner


In [3]:

# os.environ['GDAL_DATA'] = check_output('pip show rasterio | grep Location | awk \'{print $NF"/rasterio/gdal_data/"}\'', shell=True).decode().strip()
os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'


# Get the location of rasterio
rasterio_location = check_output('pip show rasterio', shell=True).decode()

# Find the line that specifies the location
gdal_data_path = None
for line in rasterio_location.splitlines():
    if line.startswith("Location:"):
        gdal_data_path = os.path.join(line.split(":", 1)[1].strip(), 'rasterio', 'gdal_data')
        break

# Set the GDAL_DATA environment variable if we found the path
if gdal_data_path:
    os.environ['GDAL_DATA'] = gdal_data_path
    print(f"GDAL_DATA has been set to: {gdal_data_path}")
else:
    print("Could not find the GDAL data path.")


GDAL_DATA has been set to: C:\Users\oltie\anaconda3\envs\ssmodel_env\Lib\site-packages\rasterio\gdal_data


In [4]:


# Set global seed for reproducibility
def set_global_seed(seed):
    random.seed(seed)  # Python's random module
    np.random.seed(seed)  # NumPy's random module
    torch.manual_seed(seed)  # PyTorch seed for CPU
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)  # PyTorch seed for current GPU
        torch.cuda.manual_seed_all(seed)  # PyTorch seed for all GPUs (if using multi-GPU)

# Prepare cuDNN for deterministic behavior
def prepare_cudnn(deterministic=True):
    torch.backends.cudnn.deterministic = deterministic  # cuDNN deterministic setting
    torch.backends.cudnn.benchmark = not deterministic  # Disable cuDNN benchmarking if deterministic

# Set the seed and configure cuDNN
SEED = 42
set_global_seed(SEED)
prepare_cudnn(deterministic=True)


### Paso 1: Descargar la Data

In [5]:
#imagenes
train_img_uri = "./data/train/img/"
train_img_name = "2023.tif"

download_blob_from_azure(train_img_name,train_img_uri)
train_iimg_uri = train_img_uri + train_img_name
#__________________________________________________________#
val_img_uri = "./data/val/img/"
val_img_name = "2023.tif"

download_blob_from_azure(val_img_name, val_img_uri)
val_img_uri = val_img_uri + val_img_name

2023.tif: 100%|##########| 362M/362M [01:07<00:00, 5.35MB/s] 


Downloaded '2023.tif' to './data/train/img/2023.tif'


2023.tif: 100%|##########| 362M/362M [00:43<00:00, 8.36MB/s] 

Downloaded '2023.tif' to './data/val/img/2023.tif'


In [8]:
train_labels_uri = "./data/train/labels/"
train_labels_name = "2023_viviendas.geojson"

download_blob_from_azure(train_labels_name,train_labels_uri)
train_labels_uri = train_labels_uri+train_labels_name


2023_viviendas.geojson: 100%|##########| 1.58M/1.58M [00:00<00:00, 6.18MB/s]

Downloaded '2023_viviendas.geojson' to './data/train/labels/2023_viviendas.geojson'


In [9]:
#labels
train_labels_uri = "./data/train/labels/"
train_labels_name = "2023_viviendas.geojson"

download_blob_from_azure(train_labels_name,train_labels_uri)
train_labels_uri = train_labels_uri+train_labels_name
#__________________________________________________________#
val_labels_uri = "./data/val/labels/"
val_labels_name = "2023_viviendas.geojson"

download_blob_from_azure(val_labels_name, val_labels_uri)
val_labels_uri = val_labels_uri + val_labels_name


2023_viviendas.geojson: 100%|##########| 1.58M/1.58M [00:00<00:00, 6.77MB/s]


Downloaded '2023_viviendas.geojson' to './data/train/labels/2023_viviendas.geojson'


2023_viviendas.geojson: 100%|##########| 1.58M/1.58M [00:00<00:00, 6.98MB/s]

Downloaded '2023_viviendas.geojson' to './data/val/labels/2023_viviendas.geojson'


In [7]:
val_labels_uri = "./data/val/labels/"
val_labels_name = "2023_viviendas.geojson"

download_blob_from_azure(val_labels_name, val_labels_uri)
val_labels_uri = val_labels_uri + val_labels_name

2023_viviendas.geojson: 100%|##########| 1.58M/1.58M [00:00<00:00, 7.21MB/s]

Downloaded '2023_viviendas.geojson' to './data/val/labels/2023_viviendas.geojson'


In [11]:
download_blob_from_azure('train_aoi.geojson', './data/train/')
download_blob_from_azure('val_aoi.geojson', './data/val/')

train_AoI_uri = './data/train/train_aoi.geojson'
val_AoI_uri = './data/val/val_aoi.geojson'

train_aoi.geojson: 100%|##########| 136k/136k [00:00<00:00, 747kB/s]


Downloaded 'train_aoi.geojson' to './data/train/train_aoi.geojson'


val_aoi.geojson: 100%|##########| 83.8k/83.8k [00:00<00:00, 264kB/s]

Downloaded 'val_aoi.geojson' to './data/val/val_aoi.geojson'


### Paso 2: Preprocesamiento: extracción de recortes para entrenamiento y test

In [12]:
# Aquí definimos las clases (tipos de objeto) presentes en nuestras anotaciones
# en este caso tenemos sólo una, que indicamos como "viviendas precarias" .
# hay un clase adicional, implícita, que es "background" -el fondo, todo lo que
# no corresponde a objetos de intéres

class_config = ClassConfig(
    names=['background', 'viviendas precarias'],
    colors=['lightgray', 'darkred'],
    null_class='background')


# el tamaño en píxeles de los recortes cuadrados
window_size = 128

# Aquí definimos algunas transformaciones a realizar a los recorte del dataset
# de entrenamiento: cambiar al azar la saturación, el brillo, rotarlos, ocultar
# algunos pixeles. Todo esto sirve para entrenar un algoritmo de detección
# más robusto a diferencias que puedan tener las futuras imágenes a las que
# se aplique
data_augmentation_transform = A.Compose([
    A.Flip(),
    A.ShiftScaleRotate(),
    A.OneOf([
        A.HueSaturationValue(hue_shift_limit=10),
        A.RandomBrightnessContrast(),
        A.RandomGamma(),
        A.RGBShift(),
        # A.ToGray(),
        # A.ToSepia(),
    ]),
    A.CoarseDropout(max_height=int(window_size/6), max_width=int(window_size/6), max_holes=4)
])


In [13]:
def create_val_ds(image_uris, class_config, AoI_file_dest_uri, val_labels_uri, window_size):
    datasets = []
    
    # Ensure image_uris is a list, even if there is only one image
    if not isinstance(image_uris, list):
        image_uris = [image_uris]

    for image_uri in image_uris:
        val_ds = SemanticSegmentationSlidingWindowGeoDataset.from_uris(
            class_config=class_config,
            aoi_uri=AoI_file_dest_uri,
            image_uri=image_uri,
            label_vector_uri=val_labels_uri,
            label_vector_default_class_id=class_config.get_class_id('viviendas precarias'),
            image_raster_source_kw=dict(allow_streaming=True, raster_transformers=[MinMaxTransformer()]),
            size=window_size,
            stride=window_size,
            transform=A.Resize(window_size, window_size)
        )
        datasets.append(val_ds)

    # Concatenate all datasets if any exist, otherwise return None
    combined_val_ds = sum(datasets[1:], datasets[0]) if datasets else None
    
    return combined_val_ds


In [14]:
# Crear el dataset de validación
val_ds = create_val_ds(
    image_uris=val_img_uri,
    class_config=class_config,
    AoI_file_dest_uri=val_AoI_uri,
    val_labels_uri=val_labels_uri,
    window_size=window_size
)

len(val_ds)


106

In [15]:
vis = SemanticSegmentationVisualizer(
    class_names=class_config.names, class_colors=class_config.colors)

x, y = val_ds[10]

vis.plot_batch(x.unsqueeze(0), y.unsqueeze(0), show=True)

In [16]:
def create_train_ds(image_uris, class_config, AoI_file_dest_uri, train_labels_uri, window_size, sample_size, data_augmentation_transform):
    datasets = []
    
    # Ensure image_uris is a list, even if there is only one image
    if not isinstance(image_uris, list):
        image_uris = [image_uris]

    for image_uri in image_uris:
        train_ds = SemanticSegmentationRandomWindowGeoDataset.from_uris(
            class_config=class_config,
            aoi_uri=AoI_file_dest_uri,
            image_uri=image_uri,
            label_vector_uri=train_labels_uri,
            label_vector_default_class_id=class_config.get_class_id('viviendas precarias'),
            image_raster_source_kw=dict(allow_streaming=True, raster_transformers=[MinMaxTransformer()]),
            size_lims=(window_size, window_size + 1),
            out_size=window_size,
            padding=100,  # Adjust the padding as needed
            max_windows=sample_size,
            transform=data_augmentation_transform
        )
        datasets.append(train_ds)
    
    # Concatenate all datasets if any exist, otherwise return None
    combined_train_ds = sum(datasets[1:], datasets[0]) if datasets else None
    
    return combined_train_ds

In [ ]:
# Crear el dataset de entrenamiento
train_ds = create_train_ds(
    image_uris=train_img_uri,
    class_config=class_config,
    AoI_file_dest_uri=train_AoI_uri,
    train_labels_uri=train_labels_uri,
    window_size=window_size,
    sample_size= len(val_ds)*10, #como las area de val es mas chica me aseguro de que train tenga 10 veces mas de imagenes para cubrir mejor el area con random windows.
    data_augmentation_transform=data_augmentation_transform
)

len(train_ds)


In [13]:
from torch.utils.data import Dataset

class LongTargetDataset(Dataset):
    def __init__(self, base_dataset):
        self.base_dataset = base_dataset

    def __len__(self):
        return len(self.base_dataset)

    def __getitem__(self, idx):
        x, y = self.base_dataset[idx]
        y = y.long()  # Ensure target is LongTensor
        return x, y

In [14]:
# Wrap train and validation datasets
train_ds = LongTargetDataset(train_ds)
val_ds = LongTargetDataset(val_ds)

In [ ]:
x, y = vis.get_batch(train_ds, 5)

vis.plot_batch(x, y, show=True)

model

In [ ]:
import torch

model = torch.hub.load(
    'AdeelH/pytorch-fpn:0.3',
    'make_fpn_resnet',
    name='resnet18',
    fpn_type='panoptic',
    num_classes=len(class_config),
    fpn_channels=128,
    in_channels=3,
    out_size=(window_size, window_size),
    pretrained=True)


In [17]:
# from rastervision.pytorch_learner.learner_config import ExternalModuleConfig

# # Definir external_loss_def para Focal Loss
# external_loss_def = ExternalModuleConfig(
#     entrypoint='FocalLoss',  # Definir el nombre de la función de entrada
#     github_repo='your/repo',  # Reemplazá con el repositorio correcto que contiene Focal Loss
#     entrypoint_kwargs={'alpha': 1.0, 'gamma': 2.0},  # Parámetros de Focal Loss
#     force_reload=False  # Puedes ajustar esto si necesitas forzar la recarga
# )

# # Configuración del solver con Focal Loss
# solver_cfg = SolverConfig(
#     batch_sz=50,
#     lr=3e-2,
#     num_epochs=10,
#     external_loss_def=external_loss_def,  # Usar la definición externa de Focal Loss
#     class_loss_weights=None  # No es necesario especificar class_loss_weights ya que Focal Loss maneja esto
# )


In [ ]:
# Set the number of epochs and the directory to save the model
n_epochs = 1000
model_folder = f"./data/models/bundle_{n_epochs}/"

# Define the batch size for training
batch_images = 100

# Configure the dataset with the class information
data_cfg = SemanticSegmentationGeoDataConfig(
    class_config=class_config,
    num_workers=0  # Set to 0 for Google Colab, to avoid multiprocessing issues
)

# Configure the solver (optimizer and scheduler)
solver_cfg = SolverConfig(
    batch_sz=batch_images,
    lr=3e-2,
    num_epochs=n_epochs,  # Specify number of epochs in the solver config
    class_loss_weights=[1.0, 10.0]  # Lower weight for background, higher for houses (nn.CrossEntropyLoss)
)

# Build the optimizer and scheduler from solver configuration
optimizer = solver_cfg.build_optimizer(model)
scheduler = solver_cfg.build_epoch_scheduler(optimizer)

# Combine everything into the learner configuration
learner_cfg = SemanticSegmentationLearnerConfig(
    data=data_cfg,
    solver=solver_cfg
)

# Instantiate the learner with the built optimizer and scheduler
learner = SemanticSegmentationLearner(
    cfg=learner_cfg,
    output_dir=model_folder,
    model=model,
    train_ds=train_ds,
    valid_ds=val_ds,
    training=True
)

# Now manually set the optimizer and scheduler into the learner if needed
learner.optimizer = optimizer
learner.scheduler = scheduler


In [ ]:
# Definir el optimizador (TODAVIA NO FUE USADO)
optimizer = learner.build_optimizer()
optimizer
#or
optimizer = solver_cfg.build_optimizer(model)
optimizer

scheduler = solver_cfg.build_epoch_scheduler(optimizer)
scheduler


In [20]:
%load_ext tensorboard

In [ ]:
logdir = f"./data/models/bundle_{n_epochs}/tb-logs"
%tensorboard --bind_all --logdir $logdir --reload_interval 10

train

In [ ]:
# Inicia el entrenamiento
learner.train(epochs=n_epochs)

In [ ]:
learner.plot_predictions(split='valid', show=True,batch_limit=5)

In [ ]:
learner.save_model_bundle()

In [ ]:
if fine_tuning:
    #BSUCAR NUEVA DATA Y HACER LOS DATASETS
    from rastervision.pytorch_learner import SemanticSegmentationLearner

    learner = SemanticSegmentationLearner.from_model_bundle(
        model_bundle_uri=f'data/models/bundle_{n_epochs}/model-bundle.zip',
        output_dir=f'data/models/bundle_{n_epochs}_extra/',
        model=model,
        train_ds=train_ds_nueva,
        valid_ds=val_ds_nueva,
        training=True,
    )
    # Inicia el entrenamiento
    learner.train(epochs=n_epochs)

BUSCAR NUEVA DATA


In [ ]:
from rastervision.pytorch_learner import SemanticSegmentationLearner

learner = SemanticSegmentationLearner.from_model_bundle(
    model_bundle_uri=f'data/models/bundle_{n_epochs}/model-bundle.zip',
    output_dir=f'data/models/bundle_{n_epochs}_extra/',
    model=model,
    train_ds=train_ds_nueva,
    valid_ds=val_ds_nueva,
    training=True,
)

In [ ]:
# Inicia el entrenamiento
learner.train(epochs=n_epochs)